In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import datetime as dt
import pickle
import numpy as np
from itertools import chain, combinations
import random
import scipy as sp
from __future__ import division
from dict_stops import *
import pandas as pd
import os
import csv

In [2]:
frame = pd.read_csv('/home/cata/Documentos/Datois/etapas_2013_abril_allyearsids_10_100000.csv')

In [3]:
if os.name == 'nt':
    path_subway_dictionary = 'C:\Users\catalina\Documents\Datois\Diccionario-EstacionesMetro.csv'
    path_csv_sequences = 'C:\Users\catalina\Documents\sequences\\'
else:
    path_subway_dictionary = '/home/cata/Documentos/Datois/Diccionario-EstacionesMetro.csv'
    path_csv_sequences = '/home/cata/Documentos/sequences/'

# Función que carga las estaciones de metro
# en un diccionario
def load_metro_dictionary():
    dict_metro = {}
    with open(path_subway_dictionary,mode='r') as infile:
        reader = csv.reader(infile,delimiter=';')
        dict_metro = {rows[5]:rows[7] for rows in reader}
    return dict_metro


In [4]:
# Función que estandariza los valores de los paraderos de subida 
# y bajada
def update_vals(row,data = load_metro_dictionary()):
    if row.par_subida in data:
        row.par_subida = data[row.par_subida]
    if row.par_bajada in data:
        row.par_bajada = data[row.par_bajada]
    return row

In [5]:
# Función que estandariza los valores de los paraderos de subida 
# y bajada
def add_vals(row,latlong,paradero,data = dict_latlong_stops):
    stop_name = row[paradero]
    if stop_name in data:
        return data[stop_name][latlong]
    else :
        return np.nan

In [6]:
def frame_config(frame):
    frame['tiempo_subida'] = pd.to_datetime(frame.tiempo_subida)
    frame['tiempo_bajada'] = pd.to_datetime(frame.tiempo_bajada)
    frame = frame.apply(update_vals, axis=1)
    frame['weekday'] = frame.tiempo_subida.dt.dayofweek
    frame['lat_subida'] = frame.apply(add_vals,args=('lat','par_subida'),axis=1)
    frame['lat_bajada'] = frame.apply(add_vals,args=('lat','par_bajada'),axis=1)
    frame['long_subida'] = frame.apply(add_vals,args=('long','par_subida'),axis=1)
    frame['long_bajada'] = frame.apply(add_vals,args=('long','par_bajada'),axis=1)
    frame = frame.sort_values(by=['id', 'tiempo_subida'])
    frame['diferencia_tiempo'] = (frame['tiempo_subida']-frame['tiempo_subida'].shift()).fillna(0)
    return frame

In [7]:
def hour_to_seconds(an_hour):
    return int(an_hour.hour*3600 + an_hour.minute *60 + an_hour.second)

In [8]:
frame = frame_config(frame)

In [9]:
frame.head()

,tiempo_subida,id,x_subida,y_subida,tipo_transporte,serviciosentidovariante,tipo_dia,nviaje,netapa,x_bajada,...,par_bajada,zona_subida,zona_bajada,adulto,weekday,lat_subida,lat_bajada,long_subida,long_bajada,diferencia_tiempo
23,2013-04-14 06:45:44,1132106,348108.0,6289153.0,BUS,T203 00R,DOMINGO,1,1,346818.0,...,E-20-190-SN-40,328.0,307.0,0.0,6,-33.526277,-33.433786,-70.635551,-70.647786,0 days 00:00:00
22,2013-04-14 07:51:52,1132106,346751.0,6299389.0,BUS,T502 00I,DOMINGO,2,1,351363.0,...,T-15-135-PO-5,307.0,188.0,0.0,6,-33.434116,-33.406027,-70.648104,-70.598251,0 days 01:06:08
21,2013-04-14 19:56:47,1132106,351368.0,6302559.0,BUS,T502 00R,DOMINGO,3,1,346763.0,...,T-4-19-NS-100,188.0,55.0,0.0,6,-33.405971,-33.432332,-70.598379,-70.648651,0 days 12:04:55
20,2013-04-14 20:15:25,1132106,346713.0,6299427.0,BUS,T203 00I,DOMINGO,3,2,348095.0,...,T-24-205-NS-20,307.0,348.0,0.0,6,-33.433463,-33.525983,-70.649060,-70.635944,0 days 00:18:38
19,2013-04-15 21:04:59,1132106,348103.0,6289191.0,BUS,T206 00R,LABORAL,4,1,346844.0,...,T-20-190-SN-35,328.0,309.0,0.0,0,-33.526277,-33.434819,-70.635551,-70.647429,1 days 00:49:34


In [32]:
from scipy.stats.mstats import mode
f = lambda x: mode(x, axis=None)[0][0]
g = lambda x: mode(x,axis=None)[1][0]
aggregations = {
    'tiempo_subida': "count"  # Calculate two results for the 'network' column with a list
    
}
a_group = frame.groupby(['par_subida','tipo_transporte']).agg(aggregations)
another_group = frame.groupby(['par_subida','par_bajada','tipo_transporte']).agg(aggregations)

In [38]:
another_group

tiempo_subida
par_subida par_bajada               tipo_transporte               
ALCANTARA  BELLAS ARTES             METRO                        2
           BELLAVISTA DE LA FLORIDA METRO                        2
           CAL Y CANTO              METRO                        1
           CIUDAD DEL NINO          METRO                       11
           CUMMING                  METRO                        2
           ECUADOR                  METRO                        5
           EL GOLF                  METRO                        2
           EL PARRON                METRO                        2
           ESCUELA MILITAR          METRO                        6
           ESTACION CENTRAL         METRO                        3
           GRUTA DE LOURDES         METRO                        1
           HOSPITAL SOTERO DEL RIO  METRO                       22
           IRARRAZAVAL              METRO                        4
           LA GRANJA                METRO                        6
           LA MONEDA                METRO                       10
           LAS MERCEDES             METRO                        3
           LAS PARCELAS             METRO                        2
           LAS REJAS                METRO                        9
           LAS TORRES               METRO                        4
           LO OVALLE                METRO                        2
           LO PRADO                 METRO                        1
           LO VIAL                  METRO                        6
           LOS DOMINICOS            METRO                        1
           LOS HEROES               METRO                       11
           LOS LEONES               METRO                       14
           LOS ORIENTALES           METRO                        1
           MACUL                    METRO                        3
           MANQUEHUE                METRO                        4
           MANUEL MONTT             METRO                        7
           MIRADOR                  METRO                        2
...                                                            ...
ZAPADORES  MIRADOR                  METRO                        1
           MONTE TABOR              METRO                        1
           PARQUE BUSTAMANTE        METRO                        2
           PARQUE OHIGGINS          METRO                        3
           PATRONATO                METRO                        5
           PEDRERO                  METRO                        2
           PEDRO DE VALDIVIA        METRO                        1
           PLAZA DE ARMAS           METRO                        5
           PLAZA MAIPU              METRO                        3
           REPUBLICA                METRO                        3
           RODRIGO DE ARAYA         METRO                        2
           RONDIZONNI               METRO                        5
           SALVADOR                 METRO                        4
           SAN ALBERTO HURTADO      METRO                        1
           SAN JOAQUIN              METRO                        1
           SAN JOSE DE LA ESTRELLA  METRO                        1
           SAN MIGUEL               METRO                       10
           SAN PABLO                METRO                        1
           SANTA ANA                METRO                       16
           SANTA ISABEL             METRO                        3
           SANTA LUCIA              METRO                        4
           SANTA ROSA               METRO                        1
           SANTIAGO BUERAS          METRO                        1
           TOBALABA                 METRO                        5
           TOESCA                   METRO                        4
           UNION LATINO AMERICANA   METRO                        2
           UNIVERSIDAD CATOLICA     METRO                        3
           UNIVERSIDAD DE CHILE  

In [39]:
another_group.max(0)

tiempo_subida    100
dtype: int64

In [54]:
another_group[another_group['tiempo_subida']>50]

tiempo_subida
par_subida              par_bajada               tipo_transporte               
CAL Y CANTO             LA CISTERNA              METRO                       69
GRECIA                  TOBALABA                 METRO                       52
HOSPITAL SOTERO DEL RIO PLAZA DE ARMAS           METRO                       59
LA CISTERNA             CAL Y CANTO              METRO                       59
                        ESCUELA MILITAR          METRO                       84
                        SANTA ANA                METRO                       51
LAS REJAS               LOS LEONES               METRO                       56
                        TOBALABA                 METRO                       79
                        UNIVERSIDAD DE CHILE     METRO                       54
MANQUEHUE               LA CISTERNA              METRO                       61
                        SANTA LUCIA              METRO                       51
                        UNIVERSIDAD DE CHILE     METRO                       54
PLAZA DE ARMAS          BELLAVISTA DE LA FLORIDA METRO                       58
                        PLAZA MAIPU              METRO                      100
PLAZA DE PUENTE ALTO    TOBALABA                 METRO                       95
PLAZA MAIPU             PLAZA DE ARMAS           METRO                       98
SANTA ANA               LA CISTERNA              METRO                       51
UNIVERSIDAD DE CHILE    LAS REJAS                METRO                       52
                        PEDRO DE VALDIVIA        METRO                       58
                        TOBALABA                 METRO                       54
VICENTE VALDES          PLAZA DE ARMAS           METRO                       55
                        TOBALABA                 METRO                       56

In [ ]:
1406 transacciones en total, de 100000